In [1]:
import pandas as pd
import numpy as np

## <font color=black>Leitura de Dados</font> 

In [2]:
# Read all files 
country = pd.read_csv('Country_Original.csv')
league = pd.read_csv('League_Original.csv')
match = pd.read_csv('Match_Original.csv')
player = pd.read_csv('Player_Original.csv')
player_attributes = pd.read_csv('Player_Attributes_Original.csv')
team = pd.read_csv('Team_Original.csv')
team_attributes = pd.read_csv('Team_Attributes_Original.csv')

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

## <font color=black>Organização e Limpeza de Dados</font> 

Pretendemos nesta secção do trabalho reduzir e tratar os dados de forma a que não exista informação irrelevante para o modelo que pretendemos construir.

### <font color=black> - Eliminação de Valores Nulos</font>

In [3]:
# Eliminar atributos relativos à posição dos jogadores
player_attributes.drop(player_attributes.columns.to_series()["potential":], axis=1, inplace = True)


In [4]:
# Eliminar duplicados caso existam
player_attributes.drop_duplicates(keep = False, inplace = True)

In [5]:
# Verficar se existe overall_ratings null
player_attributes.dropna(axis=0, how="any", subset =['overall_rating'], inplace=True)
player_attributes['overall_rating'].isnull().sum()

0

In [6]:
# Eliminar atributos relativos à posição dos jogadores
match.drop(match.columns.to_series()["home_player_X1":"away_player_Y11"], axis=1, inplace = True)
# Eliminar os seguintes atributos -> shoton, shotoff, foulcommit, card, cross, corner, possession
match.drop(match.columns.to_series()["goal":"possession"], axis=1,inplace = True)

In [7]:
#Organizar dados por datas, do passado para o presente
match['date'] = match['date'].astype('datetime64[ns]')
match = match.sort_values(by=['date'], ascending=True)

In [8]:
#Eliminar todos as linhas que não tenham nenhuma info sobre os jogadores
match = match.dropna(axis=0, how="all", subset=match.columns.to_series()["home_player_1":"away_player_11"])

In [9]:
#Eliminar todas as linhas que não tenham nenhuma odd relativa ao jogo
match.dropna(axis=0, how="all", subset=match.columns.to_series()["B365H":], inplace=True)

In [10]:
#Eliminar partidas em que não são conhecidos pelo menos 8 jogadores que jogaram (para a Equipa da Casa)
match = match.dropna(axis=0, thresh=8, subset=match.columns.to_series()["home_player_1":"home_player_11"])
#Verificação
match.loc[match.loc[:,'home_player_1':'home_player_11'].count(axis=1) < 8]

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,SJH,SJD,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA


In [11]:
#Eliminar partidas em que não são conhecidos pelo menos 8 jogadores que jogaram (para a Equipa de Fora)
match = match.dropna(axis=0, thresh=8, subset=match.columns.to_series()["away_player_1":"away_player_11"])
#Verificação
match.loc[match.loc[:,'away_player_1':'away_player_11'].count(axis=1) < 8]

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,SJH,SJD,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA


### <font color=black> - Substituição de Valores Nulos </font>

In [12]:
#Passar o formato dos atributos 'date' para formato de tempo
match['date'] = match['date'].astype('datetime64[ns]') 
player_attributes['date'] = player_attributes['date'].astype('datetime64[ns]') 
team_attributes['date'] = team_attributes['date'].astype('datetime64[ns]') 

In [13]:
#Create copy of the match and player dataframe
match_copy = match.copy()
player_copy = player.copy()
player_height = player_copy[["player_api_id", "height","weight"]].copy()

In [14]:
# Ir buscar as alturas para cada jogador a jogar a partida
for i in range(1,12):
    match_copy = match_copy.merge(player_height, how='left', left_on=['home_player_'+'%s'%i], right_on=['player_api_id'], suffixes=('', '_home_'+'%s'%i), validate="m:1") 

In [15]:
# Eliminar as colunas do player_api_id e o mudar o nome de height para height_home_1
match_copy = match_copy[match_copy.columns.drop(list(match_copy.filter(regex='player_api_id')))]
match_copy.rename(columns={"height": "height_home_1"}, inplace=True)
match_copy.rename(columns={"weight": "weight_home_1"}, inplace=True)

In [16]:
match_copy['mean_height_home']= match_copy[list(match_copy.filter(regex='height_home'))].mean(axis=1)
match_copy['mean_weight_home']= match_copy[list(match_copy.filter(regex='weight_home'))].mean(axis=1)

In [17]:
for i in range(1,12):
    match_copy = match_copy.merge(player_height, how='left', left_on=['away_player_'+'%s'%i], right_on=['player_api_id'], suffixes=('', '_away_'+'%s'%i), validate="m:1") 

In [18]:
# Eliminar as colunas do player_api_id e o mudar o nome de height para height_away_1
match_copy = match_copy[match_copy.columns.drop(list(match_copy.filter(regex='player_api_id')))]
match_copy.rename(columns={"height": "height_away_1"}, inplace=True)
match_copy.rename(columns={"weight": "weight_away_1"}, inplace=True)

In [19]:
match_copy['mean_height_away']= match_copy[list(match_copy.filter(regex='height_away'))].mean(axis=1)
match_copy['mean_weight_away']= match_copy[list(match_copy.filter(regex='weight_away'))].mean(axis=1)

In [20]:
# Eliminar colunas com alturas e pesos dos jogadores 
match_copy = match_copy[match_copy.columns.drop(list(match_copy.filter(regex='height_home_')))]
match_copy = match_copy[match_copy.columns.drop(list(match_copy.filter(regex='height_away_')))]
match_copy = match_copy[match_copy.columns.drop(list(match_copy.filter(regex='weight_home_')))]
match_copy = match_copy[match_copy.columns.drop(list(match_copy.filter(regex='weight_away_')))]

In [21]:
home_ods = [a for a in match_copy.columns if a.endswith('H')]
draw_ods = [b for b in match_copy.columns if b.endswith('D')]
away_ods = [c for c in match_copy.columns if c.endswith('A')]

In [22]:
match_copy['mean_ods_home'] = match_copy[home_ods].mean(axis=1)
match_copy['mean_ods_draw'] = match_copy[draw_ods].mean(axis=1)
match_copy['mean_ods_away'] = match_copy[away_ods].mean(axis=1)

In [23]:
#Retirar colunas das ods
odds_indices_drop = ['B365H','B365D','B365A','BWH','BWD','BWA','IWH','IWD','IWA','LBH','LBD','LBA','PSH','PSD','PSA','WHH','WHD','WHA','SJH','SJD','SJA','VCH','VCD','VCA','GBH','GBD','GBA','BSH','BSD','BSA']
match_copy.drop(columns=odds_indices_drop, inplace=True)

In [24]:
# Ir buscar os overall_ratings para cada jogador a jogar a partida
player_ratings = player_attributes[["player_api_id", "date","overall_rating"]].copy()
for i in range(1,12):
    match_copy = match_copy.merge(player_ratings, how='outer', left_on=['home_player_'+'%s'%i], right_on=['player_api_id'], suffixes=('', '_home'), validate="m:m")
    index_to_drop1 = match_copy[ (match_copy['date_home'] > match_copy['date'])].index
    match_copy.drop(index_to_drop1 , inplace=True)
    match_copy.drop_duplicates(subset= ['id'], keep='first', inplace=True) 
    match_copy.rename(columns={"overall_rating": "or_home_player_"+'%s'%i}, inplace=True)
    match_copy.drop(columns=['player_api_id','date_home'] , inplace=True)

In [25]:
# Ir buscar os overall_ratings para cada jogador a jogar a partida
for i in range(1,12):
    match_copy = match_copy.merge(player_ratings, how='outer', left_on=['away_player_'+'%s'%i], right_on=['player_api_id'], suffixes=('', '_away'), validate="m:m")
    index_to_drop1 = match_copy[ (match_copy['date_away'] > match_copy['date'])].index
    match_copy.drop(index_to_drop1 , inplace=True)
    match_copy.drop_duplicates(subset= ['id'], keep='first', inplace=True) 
    match_copy.rename(columns={"overall_rating": "or_away_player_"+'%s'%i}, inplace=True)
    match_copy.drop(columns=['player_api_id','date_away'] , inplace=True)

In [26]:
match_copy['mean_or_home']= match_copy[list(match_copy.filter(regex='or_home_player'))].mean(axis=1)
match_copy['mean_or_away']= match_copy[list(match_copy.filter(regex='or_away_player'))].mean(axis=1)
match_copy = match_copy[match_copy.columns.drop(list(match_copy.filter(regex='or_away_player_')))]
match_copy = match_copy[match_copy.columns.drop(list(match_copy.filter(regex='or_home_player_')))]

In [27]:
match_copy = match_copy[match_copy.columns.drop(list(match_copy.filter(regex='home_player')))]
match_copy = match_copy[match_copy.columns.drop(list(match_copy.filter(regex='away_player')))]

In [28]:
match_copy["dif_goals"] = match_copy.home_team_goal - match_copy.away_team_goal
match_copy = match_copy[match_copy.columns.drop(list(match_copy.filter(regex='home_team_goal')))]
match_copy = match_copy[match_copy.columns.drop(list(match_copy.filter(regex='away_team_goal')))]


In [29]:
match_copy = match_copy[match_copy.columns.drop(list(match_copy.filter(regex='country_id')))]
match_copy.head(20)

,id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,mean_height_home,mean_weight_home,mean_height_away,mean_weight_away,mean_ods_home,mean_ods_draw,mean_ods_away,mean_or_home,mean_or_away,dif_goals
25,19694.0,19694.0,2008/2009,1.0,2008-08-09,489981.0,8596.0,8548.0,182.118000,157.500000,185.650909,180.272727,6.166667,3.805556,1.462222,62.200000,71.636364,-1.0
51,19720.0,19694.0,2008/2009,13.0,2008-11-12,490055.0,9927.0,8548.0,180.340000,165.727273,183.803636,174.363636,5.666667,3.727778,1.512222,67.272727,74.090909,0.0
76,19975.0,19694.0,2009/2010,17.0,2009-12-27,658900.0,10251.0,8548.0,182.418182,166.454545,184.265455,174.909091,3.973333,3.444444,1.833333,66.545455,73.818182,-3.0
103,20022.0,19694.0,2009/2010,24.0,2010-02-10,659001.0,9927.0,8548.0,180.801818,170.000000,183.803636,178.818182,6.650000,3.883333,1.494444,64.636364,73.909091,0.0
129,20067.0,19694.0,2009/2010,31.0,2010-03-27,659045.0,9860.0,8548.0,181.725455,164.000000,182.880000,174.000000,5.672222,3.611111,1.561111,65.727273,74.000000,-3.0
157,19928.0,19694.0,2009/2010,10.0,2009-12-15,658785.0,9938.0,8548.0,184.404000,172.600000,182.649091,172.545455,3.750000,3.377778,1.926667,64.900000,73.272727,-3.0
184,19953.0,19694.0,2009/2010,14.0,2009-12-05,658878.0,8596.0,8548.0,180.109091,160.636364,182.649091,175.545455,7.055556,4.156667,1.413333,62.545455,74.818182,-2.0
213,19774.0,19694.0,2008/2009,21.0,2009-01-04,490103.0,8066.0,8548.0,180.570909,164.818182,182.187273,174.363636,7.944444,4.324444,1.376667,63.181818,74.363636,-3.0
237,20051.0,19694.0,2009/2010,29.0,2010-03-09,659035.0,8597.0,8548.0,178.816000,167.300000,182.418182,173.181818,7.555556,4.197778,1.371111,63.900000,73.909091,-2.0
267,19749.0,19694.0,2008/2009,18.0,2008-12-13,490084.0,9938.0,8548.0,182.880000,166.909091,182.880000,176.818182,4.547778,3.461111,1.731111,65.363636,74.272727,0.0


In [30]:
team_attributes_copy = team_attributes.copy()

In [31]:
colToDrop = [d for d in team_attributes_copy.columns if d.endswith('Class')]
team_attributes_copy.drop(columns=colToDrop, inplace=True)

In [32]:
team_attributes_copy.head(20)

,id,team_fifa_api_id,team_api_id,date,buildUpPlaySpeed,buildUpPlayDribbling,buildUpPlayPassing,chanceCreationPassing,chanceCreationCrossing,chanceCreationShooting,defencePressure,defenceAggression,defenceTeamWidth
0,1,434,9930,2010-02-22,60,NaN,50,60,65,55,50,55,45
1,2,434,9930,2014-09-19,52,48.0,56,54,63,64,47,44,54
2,3,434,9930,2015-09-10,47,41.0,54,54,63,64,47,44,54
3,4,77,8485,2010-02-22,70,NaN,70,70,70,70,60,70,70
4,5,77,8485,2011-02-22,47,NaN,52,53,48,52,47,47,52
5,6,77,8485,2012-02-22,58,NaN,62,45,70,55,40,40,60
6,7,77,8485,2013-09-20,62,NaN,45,40,50,55,42,42,60
7,8,77,8485,2014-09-19,58,64.0,62,56,68,57,41,42,60
8,9,77,8485,2015-09-10,59,64.0,53,51,72,63,49,45,63
9,10,614,8576,2010-02-22,60,NaN,40,45,35,55,30,70,30


In [33]:
team_attributes_copy.shape

(1458, 13)

In [34]:
# Verificação de que existem pelo menos dois parametros para avaliar a equipa
team_attributes_copy = team_attributes_copy.dropna(axis=0, thresh=2, subset=["buildUpPlaySpeed", "buildUpPlayDribbling" ,"buildUpPlayPassing"])
team_attributes_copy = team_attributes_copy.dropna(axis=0, thresh=2, subset=["chanceCreationPassing","chanceCreationCrossing","chanceCreationShooting"])
team_attributes_copy = team_attributes_copy.dropna(axis=0, thresh=2, subset=["defencePressure","defenceAggression","defenceTeamWidth"])
team_attributes_copy.shape

(1458, 13)

In [35]:
#Divide os atributos das equipas em 3 categorias : Atacante, medio e defesa
team_attributes_copy["offense_team_rates"] = team_attributes_copy[list(team_attributes_copy.filter(regex='buildUp'))].mean(axis=1)
team_attributes_copy["center_team_rates"] = team_attributes_copy[list(team_attributes_copy.filter(regex='chanceCreation'))].mean(axis=1)
team_attributes_copy["defense_team_rates"] = team_attributes_copy[list(team_attributes_copy.filter(regex='defence'))].mean(axis=1)

In [36]:
#Elimina as colunas que não vão ser utilizadas. Uma vez que as colunas novas já se encontram na tabela
team_attributes_copy = team_attributes_copy[team_attributes_copy.columns.drop(list(team_attributes_copy.filter(regex='buildUp')))]
team_attributes_copy = team_attributes_copy[team_attributes_copy.columns.drop(list(team_attributes_copy.filter(regex='chanceCreation')))]
team_attributes_copy = team_attributes_copy[team_attributes_copy.columns.drop(list(team_attributes_copy.filter(regex='defence')))]


In [37]:
#Definir as rates a inserir na tabela principal
team_ratings = team_attributes_copy[["team_api_id" ,  "date", "offense_team_rates" , "center_team_rates", "defense_team_rates"]]

In [38]:
#Merge para a tabela principal dos ratings da equipa da casa
match_copy = match_copy.merge(team_ratings, how='outer', left_on=["home_team_api_id"], right_on=['team_api_id'], suffixes=('', '_home'), validate="m:m")

In [39]:
match_copy.head()

,id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,mean_height_home,mean_weight_home,mean_height_away,mean_weight_away,mean_ods_home,mean_ods_draw,mean_ods_away,mean_or_home,mean_or_away,dif_goals,team_api_id,date_home,offense_team_rates,center_team_rates,defense_team_rates
0,19694.0,19694.0,2008/2009,1.0,2008-08-09,489981.0,8596.0,8548.0,182.118000,157.500000,185.650909,180.272727,6.166667,3.805556,1.462222,62.200000,71.636364,-1.0,8596.0,2010-02-22,70.0,66.666667,66.666667
1,19953.0,19694.0,2009/2010,14.0,2009-12-05,658878.0,8596.0,8548.0,180.109091,160.636364,182.649091,175.545455,7.055556,4.156667,1.413333,62.545455,74.818182,-2.0,8596.0,2010-02-22,70.0,66.666667,66.666667
2,19843.0,19694.0,2008/2009,31.0,2009-04-05,490167.0,8596.0,8548.0,180.109091,155.363636,181.956364,174.818182,6.961111,4.038889,1.404444,65.181818,73.090909,-1.0,8596.0,2010-02-22,70.0,66.666667,66.666667
3,20093.0,19694.0,2009/2010,35.0,2010-04-24,820489.0,8596.0,8429.0,179.832000,160.200000,181.725455,165.363636,2.231111,3.250000,3.000000,62.100000,61.727273,-1.0,8596.0,2010-02-22,70.0,66.666667,66.666667
4,19942.0,19694.0,2009/2010,12.0,2009-11-21,658867.0,8596.0,8429.0,179.185455,157.363636,181.263636,164.818182,2.133333,3.233333,3.168889,62.636364,64.363636,2.0,8596.0,2010-02-22,70.0,66.666667,66.666667


In [40]:
match_copy.shape

(123458, 23)

In [41]:
#Tratar o merge da equipa da casa
index_to_drop1 = match_copy[ (match_copy['date_home'] > match_copy['date'])].index
match_copy.drop(index_to_drop1 , inplace=True)
match_copy.drop_duplicates(subset= ['id'], keep='last', inplace=True) 
match_copy.rename(columns={"offense_team_rates": "offense_team_rates_home", "center_team_rates" : "center_team_rates_home", "defense_team_rates":"defense_team_rates_home"}, inplace=True)
match_copy.drop(columns=['date_home', 'team_api_id'] , inplace=True)

In [42]:
match_copy = match_copy.merge(team_ratings, how='outer', left_on=["away_team_api_id"], right_on=['team_api_id'], suffixes=('', '_away'), validate="m:m")

In [43]:
#Tratar o merge da equipa de fora
index_to_drop1 = match_copy[ (match_copy['date_away'] > match_copy['date'])].index
match_copy.drop(index_to_drop1 , inplace=True)
match_copy.drop_duplicates(subset= ['id'], keep='last', inplace=True) 
match_copy.rename(columns={"offense_team_rates": "offense_team_rates_away", "center_team_rates" : "center_team_rates_away", "defense_team_rates":"defense_team_rates_away"}, inplace=True)
match_copy.drop(columns=['date_away', 'team_api_id'] , inplace=True)

In [44]:
match_copy.shape

(17043, 24)

In [45]:
match_copy.head(20)

,id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,mean_height_home,mean_weight_home,mean_height_away,mean_weight_away,mean_ods_home,mean_ods_draw,mean_ods_away,mean_or_home,mean_or_away,dif_goals,offense_team_rates_home,center_team_rates_home,defense_team_rates_home,offense_team_rates_away,center_team_rates_away,defense_team_rates_away
0,20093.0,19694.0,2009/2010,35.0,2010-04-24,820489.0,8596.0,8429.0,179.832000,160.200000,181.725455,165.363636,2.231111,3.250000,3.000000,62.100000,61.727273,-1.0,70.000000,66.666667,66.666667,70.000000,66.666667,65.000000
4,21281.0,19694.0,2014/2015,8.0,2014-09-27,1726064.0,9927.0,8429.0,179.647273,167.181818,181.032727,164.545455,2.604286,3.307143,2.647143,59.909091,58.909091,-4.0,57.333333,49.666667,48.666667,59.000000,46.333333,48.333333
6,20158.0,19694.0,2010/2011,10.0,2010-10-30,840285.0,9927.0,8429.0,181.494545,165.272727,181.263636,165.818182,1.707778,3.533333,4.677778,68.545455,63.272727,-1.0,70.000000,68.333333,66.666667,70.000000,66.666667,65.000000
9,20265.0,19694.0,2010/2011,27.0,2011-02-19,840386.0,9927.0,8429.0,181.263636,169.818182,183.572727,168.909091,1.705556,3.550000,4.822222,67.545455,63.363636,1.0,70.000000,68.333333,66.666667,70.000000,66.666667,65.000000
13,21202.0,19694.0,2014/2015,30.0,2015-03-20,1726195.0,9927.0,8429.0,179.878182,167.090909,181.032727,165.545455,3.032857,3.285714,2.328571,62.454545,59.363636,4.0,57.333333,49.666667,48.666667,59.000000,46.333333,48.333333
17,21378.0,19694.0,2015/2016,22.0,2016-01-02,1990973.0,9927.0,8429.0,179.878182,171.545455,181.263636,168.272727,2.247143,3.417143,3.048571,61.272727,59.363636,0.0,55.000000,38.666667,44.000000,47.666667,65.666667,51.666667
19,20308.0,19694.0,2010/2011,33.0,2011-04-17,840423.0,10251.0,8429.0,182.649091,164.909091,178.816000,156.100000,1.613333,3.600000,5.366667,61.636364,59.400000,-1.0,49.500000,56.000000,48.333333,59.000000,46.333333,48.333333
21,20350.0,19694.0,2010/2011,5.0,2010-09-18,840255.0,10251.0,8429.0,183.341818,169.545455,182.418182,168.181818,1.787778,3.433333,4.350000,66.818182,64.090909,0.0,70.000000,66.666667,66.666667,70.000000,66.666667,65.000000
24,20181.0,19694.0,2010/2011,14.0,2010-11-20,840308.0,9860.0,8429.0,184.034545,170.818182,181.864000,165.900000,1.537778,3.883333,5.750000,67.363636,63.800000,2.0,70.000000,70.000000,66.666667,70.000000,66.666667,65.000000
29,21322.0,19694.0,2015/2016,14.0,2015-11-07,1990923.0,9860.0,8429.0,184.034545,165.909091,183.341818,173.909091,1.982857,3.481429,3.658571,64.272727,59.090909,2.0,56.000000,58.666667,52.666667,47.666667,65.666667,51.666667


In [49]:
match_copy = match_copy.round(2)

In [50]:
match_copy.head(20)

,id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,mean_height_home,mean_weight_home,mean_height_away,mean_weight_away,mean_ods_home,mean_ods_draw,mean_ods_away,mean_or_home,mean_or_away,dif_goals,offense_team_rates_home,center_team_rates_home,defense_team_rates_home,offense_team_rates_away,center_team_rates_away,defense_team_rates_away
0,20093.0,19694.0,2009/2010,35.0,2010-04-24,820489.0,8596.0,8429.0,179.83,160.20,181.73,165.36,2.23,3.25,3.00,62.10,61.73,-1.0,70.00,66.67,66.67,70.00,66.67,65.00
4,21281.0,19694.0,2014/2015,8.0,2014-09-27,1726064.0,9927.0,8429.0,179.65,167.18,181.03,164.55,2.60,3.31,2.65,59.91,58.91,-4.0,57.33,49.67,48.67,59.00,46.33,48.33
6,20158.0,19694.0,2010/2011,10.0,2010-10-30,840285.0,9927.0,8429.0,181.49,165.27,181.26,165.82,1.71,3.53,4.68,68.55,63.27,-1.0,70.00,68.33,66.67,70.00,66.67,65.00
9,20265.0,19694.0,2010/2011,27.0,2011-02-19,840386.0,9927.0,8429.0,181.26,169.82,183.57,168.91,1.71,3.55,4.82,67.55,63.36,1.0,70.00,68.33,66.67,70.00,66.67,65.00
13,21202.0,19694.0,2014/2015,30.0,2015-03-20,1726195.0,9927.0,8429.0,179.88,167.09,181.03,165.55,3.03,3.29,2.33,62.45,59.36,4.0,57.33,49.67,48.67,59.00,46.33,48.33
17,21378.0,19694.0,2015/2016,22.0,2016-01-02,1990973.0,9927.0,8429.0,179.88,171.55,181.26,168.27,2.25,3.42,3.05,61.27,59.36,0.0,55.00,38.67,44.00,47.67,65.67,51.67
19,20308.0,19694.0,2010/2011,33.0,2011-04-17,840423.0,10251.0,8429.0,182.65,164.91,178.82,156.10,1.61,3.60,5.37,61.64,59.40,-1.0,49.50,56.00,48.33,59.00,46.33,48.33
21,20350.0,19694.0,2010/2011,5.0,2010-09-18,840255.0,10251.0,8429.0,183.34,169.55,182.42,168.18,1.79,3.43,4.35,66.82,64.09,0.0,70.00,66.67,66.67,70.00,66.67,65.00
24,20181.0,19694.0,2010/2011,14.0,2010-11-20,840308.0,9860.0,8429.0,184.03,170.82,181.86,165.90,1.54,3.88,5.75,67.36,63.80,2.0,70.00,70.00,66.67,70.00,66.67,65.00
29,21322.0,19694.0,2015/2016,14.0,2015-11-07,1990923.0,9860.0,8429.0,184.03,165.91,183.34,173.91,1.98,3.48,3.66,64.27,59.09,2.0,56.00,58.67,52.67,47.67,65.67,51.67
